In [3]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from IPython.display import display

%matplotlib inline

In [4]:
data_dir = "data"
# data_types = ["chugun", "gas", "lom", "plavki", "produv", "sip"]
data_types = ["sip"]
target = "target"

mode = "train"
for i in data_types:
    print(">>>", i, mode)
    df = pd.read_csv(f"{data_dir}/{i}_{mode}.csv")
    print("— shape:", df.shape)
    display(df.head())

>>> sip train


FileNotFoundError: [Errno 2] No such file or directory: 'data/sip_train.csv'

In [325]:
def chron_features(chronom_train, chronom_test, plavki_train, plavki_test):
        for i in ["train", "test"]:
                chronom = eval(f"chronom_{i}")
                plavki = eval(f"plavki_{i}")

                chronom = (
                        pd.read_csv("data/chronom_train.csv")
                        .drop(columns="Unnamed: 0")
                        .assign(VR_KON=pd.to_datetime(chronom.VR_KON),
                                VR_NACH=pd.to_datetime(chronom.VR_NACH))

                        .assign(VR_KON=lambda x: x.VR_KON.apply(lambda x: x.replace(year=2021) if x.year == 2011 else x),
                                VR_NACH=lambda x: x.VR_NACH.apply(lambda x: x.replace(year=2021) if x.year == 2011 else x))

                        .assign(duration=lambda x: (x.VR_KON - x.VR_NACH).dt.total_seconds(),
                                O2=lambda x: x.O2.fillna(0))
                )
                plavki = (
                        pd.read_csv("data/plavki_train.csv")
                .assign(plavka_VR_NACH=pd.to_datetime(plavki.plavka_VR_NACH),
                        plavka_VR_KON=pd.to_datetime(plavki.plavka_VR_KON))
                [["NPLV", "plavka_VR_NACH", "plavka_VR_KON"]]
                )
                df = (
                        chronom.merge(plavki, on="NPLV", how="inner")
                        .sort_values(["plavka_VR_NACH", "VR_NACH", "O2"], ascending=[True, True, False])
                        .drop_duplicates(subset=['NPLV', 'TYPE_OPER', 'NOP', 'VR_NACH', 'VR_KON'], keep="first")

                        .assign(sec_from_start=lambda x: (x.VR_NACH - x.plavka_VR_NACH).dt.total_seconds(),
                                sec_till_end=lambda x: (x.plavka_VR_KON - x.VR_KON).dt.total_seconds())

                        .query("sec_till_end > 0")

                        .assign(prep=lambda x: np.where(x.sec_from_start < 0, 1, 0))
                        .groupby(["NPLV", "TYPE_OPER", "prep"], as_index=False, sort=False)
                        
                        .agg(O2=("O2", "sum"), 
                                        total_duration=("duration", "sum"),
                                        min_duration=("duration", "min"),
                                        max_duration=("duration", "max"),
                                        total_operations=("NOP", "count"))
                        .assign(TYPE_OPER=lambda x: x.TYPE_OPER + "_" + x.prep.astype("str"))
                        .drop(columns="prep")
                        .pivot(index="NPLV", 
                                columns="TYPE_OPER",
                                values=["O2", "total_duration", "min_duration", "max_duration", "total_operations"])
                                .fillna(0)          
                )
                df.columns = df.columns.map('_'.join).str.strip('_')
                if i=="train":
                        train = df
                else:
                        test = df
        return train, test, list(train.columns), []

In [18]:
chugun = pd.read_csv("../data/chugun_train.csv")[["NPLV", "VES"]]
sip = (
    pd.read_csv("../data/sip_train.csv")
    .merge(chugun, on="NPLV", how="inner")
    .assign(type_syp=lambda x: x.VDSYP.astype("str") + "_" + x.NMSYP)
    .drop(columns=["VDSYP", "NMSYP"])

    .groupby("NPLV")
    .agg(
        min_mass=("VSSYP", "min"),
        m_mass=("VSSYP", lambda x: x * 100),
        mi_mass=("VSSYP", "min")
    )
)
df.head()


ValueError: Must produce aggregated value

array([346, 408, 171, 442, 104, 119, 397])

In [281]:
df.pivot(
    index="NPLV", 
    columns="TYPE_OPER",
    values=["O2", "total_duration", "min_duration", "max_duration", "total_operations"]
).reset_index()

NPLV             O2                                        \
TYPE_OPER         вн.пл.прост._0 межпл.прост._0 межпл.прост._1  опер_0   
0          510008            0.0            NaN            0.0  2909.0   
1          510009            0.0            NaN            0.0  2182.0   
2          510010            NaN            NaN            0.0  2841.0   
3          510011            NaN            NaN            0.0    10.0   
4          510012            NaN            NaN            0.0  3225.0   
...           ...            ...            ...            ...     ...   
2058       512318            0.0            NaN            0.0  1917.0   
2059       512319            0.0            NaN            0.0  1660.0   
2060       512320            0.0            NaN            0.0     0.0   
2061       512321            0.0            NaN            0.0   371.0   
2062       512322            0.0            NaN            0.0   665.0   

          total_duration                                         min_duration  \
TYPE_OPER вн.пл.прост._0 межпл.прост._0 межпл.прост._1  опер_0 вн.пл.прост._0   
0                  246.0            NaN          840.0  2489.0          246.0   
1                  922.0            NaN         1140.0  2987.0          302.0   
2                    NaN            NaN          594.0  2796.0            NaN   
3                    NaN            NaN         1542.0  2966.0            NaN   
4                    NaN            NaN          601.0  2836.0            NaN   
...                  ...            ...            ...     ...            ...   
2058               932.0            NaN          747.0  3064.0          179.0   
2059              1314.0            NaN         1734.0  3783.0         1314.0   
2060               438.0            NaN          859.0  3150.0          438.0   
2061               566.0            NaN          704.0  3654.0          104.0   
2062              2394.0            NaN          520.0  4842.0         2394.0   

           ...                         max_duration                 \
TYPE_OPER  ... межпл.прост._1 опер_0 вн.пл.прост._0 межпл.прост._0   
0          ...           46.0   16.0          246.0            NaN   
1          ...           54.0   31.0          620.0            NaN   
2          ...           55.0  133.0            NaN            NaN   
3          ...           42.0   17.0            NaN            NaN   
4          ...           43.0   54.0            NaN            NaN   
...        ...            ...    ...            ...            ...   
2058       ...           94.0    0.0          753.0            NaN   
2059       ...           33.0    0.0         1314.0            NaN   
2060       ...           62.0    0.0          438.0            NaN   
2061       ...           38.0    0.0          462.0            NaN   
2062       ...           18.0    0.0         2394.0            NaN   

                                 total_operations                 \
TYPE_OPER межпл.прост._1  опер_0   вн.пл.прост._0 межпл.прост._0   
0                  424.0  1170.0              1.0            NaN   
1                  574.0  1230.0              2.0            NaN   
2                  301.0  1226.0              NaN            NaN   
3                  775.0  1058.0              NaN            NaN   
4                  305.0  1239.0              NaN            NaN   
...                  ...     ...              ...            ...   
2058               377.0  1063.0              2.0            NaN   
2059               870.0  1415.0              1.0            NaN   
2060               434.0  1058.0              1.0            NaN   
2061               355.0  1169.0              2.0            NaN   
2062               263.0  2446.0              1.0            NaN   

                                 
TYPE_OPER межпл.прост._1 опер_0  
0                    5.0    6.0  
1                    6.0    6.0  
2                    4.0    6.0  
3                    4.0    7.

In [120]:
df["TYPE_OPER"] + "__" + df["NOP"]

1             межпл.прост.__межпл. простой
2          межпл.прост.__Осмотр конвертера
3        межпл.прост.__Наведение гарнисажа
4                 межпл.прост.__Слив шлака
5              межпл.прост.__Отсутствие O2
                       ...                
35660                 опер__Выпуск металла
35661                  опер__Отсечка шлака
35662                опер__Прерыв. выпуска
35663                     опер__Слив шлака
35649         межпл.прост.__межпл. простой
Length: 32581, dtype: object

In [122]:
df.NOP.unique()

array(['межпл. простой', 'Осмотр конвертера', 'Наведение гарнисажа',
       'Слив шлака', 'Отсутствие O2', 'Завалка лома', 'Нагрев лома',
       'Заливка чугуна', 'Продувка', 'Повалка', 'Ожидание стальковша',
       'Выпуск металла', 'Прерыв. выпуска', 'Вхождение в гр.МНЛЗ',
       'Обрыв горловины', 'Додувка на t', 'Наложение продувки',
       'Охл. азотом', 'Подварка  футеровки', 'Отсутствие чугуна',
       'Отсечка шлака', 'Додувка на C', 'Неиспр. механ. обор.',
       'Неиспр. электр. обор', 'Ремонт летки', 'Замена фурмы',
       'Отсутствие мет.шихты', 'ППР', 'Полусухое торкрет.',
       'Ожидание шл.чаш', 'Отсут. своб.разл.пл.', 'Замер положения фурм',
       'Неиспр. энерг. обор', 'Неиспр. АСУ и КИПиА', 'Скач. шлака'],
      dtype=object)

In [125]:
df.NOP.value_counts().head(50)

Слив шлака              4264
межпл. простой          4125
Повалка                 3356
Выпуск металла          2065
Завалка лома            2063
Заливка чугуна          2063
Продувка                2063
Наведение гарнисажа     2027
Прерыв. выпуска         2015
Осмотр конвертера       1853
Нагрев лома             1839
Отсечка шлака            860
Отсутствие чугуна        630
Додувка на t             557
Додувка на C             531
Обрыв горловины          434
Наложение продувки       327
Вхождение в гр.МНЛЗ      322
Полусухое торкрет.       182
Отсутствие O2            155
Неиспр. электр. обор     149
Охл. азотом              143
Подварка  футеровки      141
Ожидание стальковша       79
Замер положения фурм      72
Неиспр. механ. обор.      64
Отсутствие мет.шихты      49
Замена фурмы              36
ППР                       34
Ожидание шл.чаш           30
Отсут. своб.разл.пл.      23
Ремонт летки              13
Неиспр. энерг. обор       11
Неиспр. АСУ и КИПиА        5
Скач. шлака   

In [254]:
df[df.NPLV==510051].sort_values(["plavka_VR_NACH", "VR_NACH"])

,NPLV,TYPE_OPER,NOP,VR_NACH,VR_KON,O2,duration,plavka_VR_NACH,plavka_VR_KON,sec_from_start,sec_till_end
584,510051,межпл.прост.,межпл. простой,2021-01-03 04:12:12,2021-01-03 04:17:04,0.0,292.0,2021-01-03 04:17:04,2021-01-03 05:13:40,-292.0,3396.0
585,510051,межпл.прост.,Осмотр конвертера,2021-01-03 04:12:22,2021-01-03 04:13:59,0.0,97.0,2021-01-03 04:17:04,2021-01-03 05:13:40,-282.0,3581.0
586,510051,межпл.прост.,Наведение гарнисажа,2021-01-03 04:13:59,2021-01-03 04:16:02,0.0,123.0,2021-01-03 04:17:04,2021-01-03 05:13:40,-185.0,3458.0
587,510051,межпл.прост.,Слив шлака,2021-01-03 04:16:02,2021-01-03 04:16:13,0.0,11.0,2021-01-03 04:17:04,2021-01-03 05:13:40,-62.0,3447.0
588,510051,межпл.прост.,Наведение гарнисажа,2021-01-03 04:16:13,2021-01-03 04:17:04,0.0,51.0,2021-01-03 04:17:04,2021-01-03 05:13:40,-51.0,3396.0
589,510051,опер,Завалка лома,2021-01-03 04:17:04,2021-01-03 04:22:04,0.0,300.0,2021-01-03 04:17:04,2021-01-03 05:13:40,0.0,3096.0
590,510051,межпл.прост.,Слив шлака,2021-01-03 04:17:04,2021-01-03 04:17:12,0.0,8.0,2021-01-03 04:17:04,2021-01-03 05:13:40,0.0,3388.0
592,510051,опер,Нагрев лома,2021-01-03 04:22:04,2021-01-03 04:28:34,2013.0,390.0,2021-01-03 04:17:04,2021-01-03 05:13:40,300.0,2706.0
593,510051,вн.пл.прост.,Наложение продувки,2021-01-03 04:28:35,2021-01-03 04:34:41,0.0,366.0,2021-01-03 04:17:04,2021-01-03 05:13:40,691.0,2339.0
594,510051,опер,Заливка чугуна,2021-01-03 04:34:41,2021-01-03 04:38:17,0.0,216.0,2021-01-03 04:17:04,2021-01-03 05:13:40,1057.0,2123.0
